## Merging on key

How could we load the temeprature sensor data from L12, using date as index. Then, find the historic temperatures from climate.gc.ca in the same date range and merge the two datasets based on date?

In [2]:
import numpy as np
import pandas as pd

In [ ]:
pd.read_csv()

### Load sensor data

In [4]:
df = pd.read_csv('../L12-code/temp.csv', 
                header=None, 
                names=['date', 'temp'], 
                skiprows=4, 
                parse_dates=[0,], 
                date_parser=pd.to_datetime, 
                index_col=[0,])
df.head()

,temp
date,
2019-08-26 21:00:00,22.428
2019-08-26 20:45:00,22.659
2019-08-26 20:30:00,22.906
2019-08-26 20:15:00,23.368
2019-08-26 20:00:00,24.340


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 673 entries, 2019-08-26 21:00:00 to 2019-08-19 21:00:00
Data columns (total 1 columns):
temp    673 non-null float64
dtypes: float64(1)
memory usage: 10.5 KB


In [6]:
df.describe(include='all')

,temp
count,673.000000
mean,28.005504
std,2.663926
min,22.428000
25%,26.377000
50%,27.582000
75%,29.204000
max,39.388000


Because this is a Series object with datetime objects as index items, we can use `resample()` to create daily `'D'` averages.

In [7]:
df.resample('D').mean()

,temp
date,
2019-08-19,29.317667
2019-08-20,28.003146
2019-08-21,27.705844
2019-08-22,27.660521
2019-08-23,29.040990
2019-08-24,27.348052
2019-08-25,26.462729
2019-08-26,29.866459


### Load historical data
This is code from Assignment06.

Since the data in the sensor dataset spans a single week in August 2019, all we need is download the corresponding month.

In [12]:
url_template = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID={station}&Year={year}&Month={month}&Day=14&timeframe=1&submit=Download+Data"
# replace variables in string with actual values
url = url_template.format(station=50430, year=2019, month=8)
url

'https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=50430&Year=2019&Month=8&Day=14&timeframe=1&submit=Download+Data'

In [15]:
# read data into dataframe, use headers and set Date/Time column as index
weather_data = pd.read_csv(url, index_col='Date/Time', parse_dates=True)
weather_data.columns = [col.replace('\xb0', '') for col in weather_data.columns]
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 744 entries, 2019-08-01 00:00:00 to 2019-08-31 23:00:00
Data columns (total 27 columns):
Longitude (x)          744 non-null float64
Latitude (y)           744 non-null float64
Station Name           744 non-null object
Climate ID             744 non-null int64
Year                   744 non-null int64
Month                  744 non-null int64
Day                    744 non-null int64
Time                   744 non-null object
Temp (C)               744 non-null float64
Temp Flag              0 non-null float64
Dew Point Temp (C)     744 non-null float64
Dew Point Temp Flag    0 non-null float64
Rel Hum (%)            744 non-null int64
Rel Hum Flag           0 non-null float64
Wind Dir (10s deg)     744 non-null int64
Wind Dir Flag          0 non-null float64
Wind Spd (km/h)        744 non-null int64
Wind Spd Flag          0 non-null float64
Visibility (km)        744 non-null float64
Visibility Flag        0 non-null float64
Stn Pr

In [16]:
weather_data.head()

,Longitude (x),Latitude (y),Station Name,Climate ID,Year,Month,Day,Time,Temp (C),Temp Flag,...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
Date/Time,,,,,,,,,,,,,,,,,,,,,
2019-08-01 00:00:00,-114.01,51.12,CALGARY INTL A,3031092,2019,8,1,00:00,13.3,NaN,...,NaN,24.1,NaN,88.95,NaN,NaN,NaN,NaN,NaN,NaN
2019-08-01 01:00:00,-114.01,51.12,CALGARY INTL A,3031092,2019,8,1,01:00,13.5,NaN,...,NaN,24.1,NaN,88.97,NaN,NaN,NaN,NaN,NaN,NaN
2019-08-01 02:00:00,-114.01,51.12,CALGARY INTL A,3031092,2019,8,1,02:00,14.1,NaN,...,NaN,24.1,NaN,89.00,NaN,NaN,NaN,NaN,NaN,Clear
2019-08-01 03:00:00,-114.01,51.12,CALGARY INTL A,3031092,2019,8,1,03:00,13.6,NaN,...,NaN,24.1,NaN,89.03,NaN,NaN,NaN,NaN,NaN,NaN
2019-08-01 04:00:00,-114.01,51.12,CALGARY INTL A,3031092,2019,8,1,04:00,14.1,NaN,...,NaN,8.1,NaN,89.05,NaN,NaN,NaN,NaN,NaN,Fog


### Merge on datetime index

In [ ]:
pd.merge()

We merge the sensor Series in `df` with the temperature column from the historical data by using the index in each dataset as the 'merge key'.  

Note that historical data has hourly information, whereas sensor data has 15min intervals.

By default, this method performs an 'inner' join:
>inner: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.

which is OK here.

In [20]:
tdf = pd.merge(df, weather_data['Temp (C)'], left_index=True, right_index=True, sort=True)
tdf

,temp,Temp (C)
2019-08-19 21:00:00,27.103,15.1
2019-08-19 22:00:00,30.069,14.6
2019-08-19 23:00:00,29.729,13.9
2019-08-20 00:00:00,28.988,13.7
2019-08-20 01:00:00,29.575,12.3
...,...,...
2019-08-26 17:00:00,34.060,16.6
2019-08-26 18:00:00,37.869,16.3
2019-08-26 19:00:00,34.725,14.4
2019-08-26 20:00:00,24.340,11.6


Get daily averages for both temperatures.

In [21]:
tdf.resample('D').mean()

,temp,Temp (C)
2019-08-19,28.967000,14.533333
2019-08-20,28.053375,17.633333
2019-08-21,27.764167,21.416667
2019-08-22,27.967917,16.470833
2019-08-23,29.120083,16.791667
2019-08-24,27.305542,15.887500
2019-08-25,25.840375,12.800000
2019-08-26,29.824227,12.554545


Check if there is any correlation between equipment and outdoor temperature.

In [23]:
tdf.corr(method='spearman')

,temp,Temp (C)
temp,1.00000,0.14136
Temp (C),0.14136,1.00000


## Groupby
In a temperature dataset spanning one year, obtained from climate.gc.ca, how could we get monthly average temparatures using group-by?

### Get the data 

In [24]:
url_template = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID={station}&Year={year}&Month={month}&Day=14&timeframe=2&submit=Download+Data"
# replace variables in string with actual values
url = url_template.format(station=50430, year=2018, month=8)
url

'https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=50430&Year=2018&Month=8&Day=14&timeframe=2&submit=Download+Data'

In [25]:
# read data into dataframe, use headers and set Date/Time column as index
wd_2018 = pd.read_csv(url, index_col='Date/Time', parse_dates=True)
wd_2018.columns = [col.replace('\xb0', '') for col in wd_2018.columns]
wd_2018.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 365 entries, 2018-01-01 to 2018-12-31
Data columns (total 30 columns):
Longitude (x)                365 non-null float64
Latitude (y)                 365 non-null float64
Station Name                 365 non-null object
Climate ID                   365 non-null int64
Year                         365 non-null int64
Month                        365 non-null int64
Day                          365 non-null int64
Data Quality                 0 non-null float64
Max Temp (C)                 363 non-null float64
Max Temp Flag                13 non-null object
Min Temp (C)                 356 non-null float64
Min Temp Flag                13 non-null object
Mean Temp (C)                356 non-null float64
Mean Temp Flag               13 non-null object
Heat Deg Days (C)            356 non-null float64
Heat Deg Days Flag           13 non-null object
Cool Deg Days (C)            356 non-null float64
Cool Deg Days Flag           13 non-null obje

### We could use resample and `'M'` for monthly

In [26]:
wd_2018.resample('M').mean()

,Longitude (x),Latitude (y),Climate ID,Year,Month,Day,Data Quality,Max Temp (C),Min Temp (C),Mean Temp (C),Heat Deg Days (C),Cool Deg Days (C),Total Rain (mm),Total Snow (cm),Total Precip (mm),Snow on Grnd (cm),Dir of Max Gust (10s deg)
Date/Time,,,,,,,,,,,,,,,,,
2018-01-31,-114.01,51.12,3031092.0,2018.0,1.0,16.0,NaN,-0.070968,-12.596667,-6.236667,24.236667,0.000000,0.000000,0.354839,0.274194,7.064516,26.826087
2018-02-28,-114.01,51.12,3031092.0,2018.0,2.0,14.5,NaN,-5.875000,-17.848148,-11.644444,29.644444,0.000000,0.000000,1.546429,1.350000,12.928571,29.111111
2018-03-31,-114.01,51.12,3031092.0,2018.0,3.0,16.0,NaN,0.138710,-10.383871,-5.141935,23.141935,0.000000,0.043333,1.351613,1.138710,10.161290,22.437500
2018-04-30,-114.01,51.12,3031092.0,2018.0,4.0,15.5,NaN,7.380000,-4.223333,1.586667,16.413333,0.000000,0.282759,0.820000,0.990000,2.272727,20.240000
2018-05-31,-114.01,51.12,3031092.0,2018.0,5.0,16.0,NaN,21.125806,7.410345,14.334483,3.948276,0.282759,1.048387,0.000000,1.048387,NaN,19.133333
2018-06-30,-114.01,51.12,3031092.0,2018.0,6.0,15.5,NaN,21.613333,8.700000,15.221429,3.085714,0.307143,2.306897,0.000000,2.256667,NaN,23.620690
2018-07-31,-114.01,51.12,3031092.0,2018.0,7.0,16.0,NaN,24.932258,10.625806,17.800000,1.319355,1.119355,1.232143,0.000000,1.160000,NaN,19.200000
2018-08-31,-114.01,51.12,3031092.0,2018.0,8.0,16.0,NaN,24.119355,9.990323,17.080645,2.151613,1.232258,1.603226,0.000000,1.603226,NaN,19.629630
2018-09-30,-114.01,51.12,3031092.0,2018.0,9.0,15.5,NaN,11.853333,2.683333,7.286667,10.713333,0.000000,1.180000,0.060000,1.240000,0.000000,23.450000


### Using groupby 
There is a 'Month' column, so we can use `groupby()` on this column and then ask for the mean in each sub-group:

In [31]:
wd_2018['Mean Temp (C)'].groupby(wd_2018['Month']).mean()

Month
1     -6.236667
2    -11.644444
3     -5.141935
4      1.586667
5     14.334483
6     15.221429
7     17.800000
8     17.080645
9      7.286667
10     4.093333
11    -0.227586
12    -2.860000
Name: Mean Temp (C), dtype: float64